### Instructions

For the period 12/31/2019 - 12/31/2020:
    A. Compute daily excess return levels based on comparisons to risk free (3Mo T-bill)
    B. Compute log cumulative excess return over the same period

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)

### Helper function definitions

In [2]:
#Define helper functions for analysis
def process_tbill(df):
    df['DTB3'] = df['DTB3'].replace('.', np.NaN)
    df['DTB3'] = df['DTB3'].astype(float)
    df['DATE'] = pd.to_datetime(df['DATE'])
    mask = (df['DATE'] >= '2019-12-30') & (df['DATE'] <= '2020-12-31')
    df = df.loc[mask]
    df = df.rename(columns={'DTB3':'price', 'DATE':'date'})
    return df

def process_oil(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.rename(columns={'Cushing, OK WTI Spot Price FOB (Dollars per Barrel)':'price', 'Date':'date'})
    mask = (df['date'] >= '2019-12-30') & (df['date'] <= '2020-12-31')
    df = df.loc[mask]
    return df

def calc_daily_return(df):
    df['daily_return'] = df['price']/df['price'].shift(1)-1
    return df

### Data Wrangling

In [3]:
#Load datasets
#Historical Daily Oil Prices
df_oil = pd.read_csv('RWTCd.csv')
df_oil = process_oil(df_oil)

#3 Month Treasury Bill
df_tbill = pd.read_csv('3MO.csv')
df_tbill = process_tbill(df_tbill)

#check for missing dates in oil prices
date_range = pd.date_range(start="2019-12-31", end="2020-12-31", freq='B') #range of business days in desired window
missing_dates = date_range.difference(df_oil['date'])

print(missing_dates,'\n') #missing oil price dates are holidays, remove from treasury bill data

#an additional date (2020-08-31) is missing from tbill, add in and interpolate missing price value
print(date_range.difference(df_tbill['date'])) 

df_tbill = df_tbill.loc[~df_tbill['date'].isin(missing_dates)] #remove dates from tbill

temp = {'date': pd.Timestamp('2020-08-31'), 'price': np.nan} #add in additional missing date to tbill, sort dates again
df_tbill = df_tbill.append(temp, ignore_index=True)
df_tbill = df_tbill.sort_values(by="date")

#interpolate price for any NaN values in tbill data (a few missing values are NaN as well as missing date just added)
df_tbill['price'] = df_tbill['price'].interpolate()

DatetimeIndex(['2020-01-01', '2020-01-20', '2020-02-17', '2020-04-10',
               '2020-05-25', '2020-07-03', '2020-09-07', '2020-11-26',
               '2020-11-27', '2020-12-25'],
              dtype='datetime64[ns]', freq=None) 

DatetimeIndex(['2020-08-31'], dtype='datetime64[ns]', freq=None)


### Calculations
##### Formulas

$Daily Return = \frac{Todays Price}{YesterdaysPrice} -1$                    (   $ r_t = \frac{p_t}{p_{t-1}} -1$   )

$Excess Return = Return - Riskfree Return$

$Log Returns = ln(1+return)$

$Log Cumulative = sum(Log Returns)$

In [4]:
#Calculate daily returns for oil and tbill
df_oil = calc_daily_return(df_oil)
df_tbill = calc_daily_return(df_tbill)

In [5]:
#Calculate Daily Excess Return levels (compare oil to riskfree tbill)
result = df_oil.merge(df_tbill, on='date', suffixes=('_oil','_tbill')) #combine oil and tbill into one dataframe

#calculate excess return (oil return - tbill return)
result['daily_excess_return'] = result['daily_return_oil']-result['daily_return_tbill'] 

result = result[['date','daily_return_oil','daily_return_tbill', 'daily_excess_return']]

In [6]:
#Calculate the Log Cumulative Excess Return over same period
#log return of each, add values
# np.log

df_oil['log_return'] = np.log(1+df_oil['daily_return'])
oil_log_cumulative = df_oil['log_return'].sum()

df_tbill['log_return'] = np.log(1+df_tbill['daily_return'])
tbill_log_cumulative = df_tbill['log_return'].sum()

log_cumulative_excess_return = oil_log_cumulative-tbill_log_cumulative #calculate Log Cumulative Excess Return

C:\Users\Zach\anaconda3\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


### Results

In [7]:
# Log Cumulative Excess Return
print(f'Log cumulative excess return : {log_cumulative_excess_return}')

Log cumulative excess return : 4.192300679076308


In [8]:
# Daily Excess Return Levels
result.head()

,date,daily_return_oil,daily_return_tbill,daily_excess_return
0,2019-12-30,NaN,NaN,NaN
1,2019-12-31,-0.008433,-0.012987,0.004554
2,2020-01-02,0.000491,-0.006579,0.007070
3,2020-01-03,0.029917,-0.013245,0.043162
4,2020-01-06,0.004286,0.026846,-0.022560


##### Save result to csv - uncomment cell below to save

In [9]:
# result.to_csv('daily_excess_results.csv')